In [1]:
#import geopandas as gp
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.optimize import least_squares
from scipy.interpolate import interp1d
from smrt.core.globalconstants import PERMITTIVITY_OF_AIR
from smrt import sensor_list, make_soil, make_model, make_snowpack
from matplotlib import pyplot as plt
import matplotlib

font = {'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)
plt.rcParams["axes.labelsize"] = 22
plt.rcParams["axes.labelweight"] = 'bold'
plt.rcParams['xtick.labelsize']=16
plt.rcParams['ytick.labelsize']=16

pd.set_option('use_inf_as_na', True)

from radarfunc import to_dB


In [2]:
soil_params = [2.7, 0.75, 0.011]
print(soil_params[1])

0.75


In [3]:
background = pd.read_json('TVC_Arctic_Min_2022.json')

background

ValueError: Expected object or value

In [82]:
def smrtSim(f, theta, thickness, density, temperature, p_ex, kappa_ws, kappa_dh, grain_type, soil_params, bands=['vv']):
    
    '''
    f: frequency (in Hz)
    theta: incidence angle (in degrees)
    thickness: list of snow layer thicknesses (in m)
    density: list of snow layer densities (in kg/m3)
    temperature: list of snow layer temperatures (in K)
    p_ex: list of snow layer exponential correlation lengths (in m)
    kappa_ws: polydispersity for the rounded grain snow type (Arctic wind slab)
    kapp_dh: polydispersity for the depth hoar snow grain type
    grain_type: classified snow layer grain types (R: rounded grains; H: depth hoar)
    soil_params: list of background soil properties
        epsr: real part of the soil permittivity
        epsi: imaginary part of the soil permittivity
        mss: mean square slope soil roughness parameter
    bands: polarizations
    '''
    
    #Modeling theories to use in SMRT
    # model = make_model("iba", "dort", rtsolver_options = {'m_max': 2, 'error_handling':'nan', 'phase_normalization' : False})
    
    model = make_model("iba", "dort", rtsolver_options = {'error_handling':'nan', 'phase_normalization' : True, 
                                                         'diagonalization_method':'shur_forcedtriu'})
    
    #Initializing the modeled sigma-nought values
    sim_sig0 = {}
    
    #Creating the substrate under the snowpack
    '''
    Roughness model: Geometrical Optics Backscatter model
    Permittivity model: static complexe permittivity values to optimize
    Mean_square_slope: Mean Square Slope calculated previously (mss, no units)
    Temperature: Soil temperature (in K)
    '''
    sub = make_soil('geometrical_optics', 
                    permittivity_model = complex(soil_params[0],soil_params[1]), 
                    mean_square_slope=soil_params[2], 
                    temperature = 253.7)
    
    # kappa = np.array(1 * len(grain_type))
    
    kappa = np.array([kappa_ws] * len(grain_type))
    kappa[np.where(grain_type=='H')[0]] = kappa_dh
    
    #Creating the snowpack to simulate with the substrate
    sp = make_snowpack(thickness=thickness, 
                       microstructure_model='exponential',
                       density= density,
                       temperature=temperature,
                       ice_permittivity_model=None,
                       background_permittivity_model=PERMITTIVITY_OF_AIR,
                       liquid_water=0, salinity=0, 
                       corr_length = kappa * p_ex,
                       substrate = sub)

    #Initializing the outputs dataframe
    sim_sig0=pd.DataFrame({'theta': [], 'sig0': []})

    sensor  = sensor_list.active(f, theta)

    sigma_nought = model.run(sensor, sp, parallel_computation=True)
    
    for tet in theta:
    
        for band in bands:

            sim_sig0 = pd.concat([sim_sig0,pd.DataFrame({'theta': [np.array(tet)], 
                                                         'pol':[band],
                                                         'sig0': [sigma_nought.sigma_as_dataframe().loc[
                                                                  tet,band[0].upper(),band[1].upper()].sigma],
                                                         'other_data':[sigma_nought.other_data]})], 
                                 ignore_index=True)

    return sim_sig0

In [83]:
sim_sig0={}

for i in range(2):
    
    sim_sig0[i]={}

    sim_sig0[i] = smrtSim(13.25e9, [40], 
                                  np.array(background['thickness'][i]), 
                                  np.array(background['density'][i]), 
                                  np.array(background['temperature'][i]), 
                                  np.array(background['p_ex'][i]),
                                  0.75,1.2,
                                  np.array(background['grain_type'][i]),
                                  soil_params)
    print(i)


0
1


In [84]:
# Create an empty dictionary to store the Arctic SVS2-Crocus dB output
dB_output_Arctic = {}

# Loop through the rows in sim_sig0
for b in range(len(sim_sig0)):
    dB_output_Arctic[b] = to_dB(sim_sig0[b]['sig0'])

# Print the dB_output dictionary to verify the changes
print(dB_output_Arctic)

{0: 0   -5.628953
Name: sig0, dtype: float64, 1: 0   -4.738867
Name: sig0, dtype: float64}


In [61]:
# Create an empty dictionary to store the Default SVS2-Crocus dB output
dB_output_Default = {}

# Loop through the rows in sim_sig0
for b in range(len(sim_sig0)):
    dB_output_Default[b] = to_dB(sim_sig0[b]['sig0'])

# Print the dB_output dictionary to verify the changes
print(dB_output_Default)

{0: 0   -5.946862
Name: sig0, dtype: float64, 1: 0   -6.350994
Name: sig0, dtype: float64, 2: 0   -6.972174
Name: sig0, dtype: float64, 3: 0   -5.843008
Name: sig0, dtype: float64, 4: 0   -6.885708
Name: sig0, dtype: float64, 5: 0   -5.756196
Name: sig0, dtype: float64, 6: 0   -5.338105
Name: sig0, dtype: float64, 7: 0   -6.188963
Name: sig0, dtype: float64, 8: 0   -6.737472
Name: sig0, dtype: float64, 9: 0   -5.618316
Name: sig0, dtype: float64, 10: 0   -7.096269
Name: sig0, dtype: float64, 11: 0   -5.457205
Name: sig0, dtype: float64, 12: 0   -6.118293
Name: sig0, dtype: float64, 13: 0   -5.059742
Name: sig0, dtype: float64, 14: 0   -4.814962
Name: sig0, dtype: float64, 15: 0   -5.353105
Name: sig0, dtype: float64, 16: 0   -6.458998
Name: sig0, dtype: float64, 17: 0   -5.879387
Name: sig0, dtype: float64, 18: 0   -5.616926
Name: sig0, dtype: float64, 19: 0   -6.337872
Name: sig0, dtype: float64, 20: 0   -5.072998
Name: sig0, dtype: float64, 21: 0   -6.821397
Name: sig0, dtype: float6

In [22]:
# Create an empty dictionary to store the Pit dB output
dB_output_Pit = {}

# Loop through the rows in sim_sig0
for b in range(len(sim_sig0)):
    dB_output_Pit[b] = to_dB(sim_sig0[b]['sig0'])

# Print the dB_output dictionary to verify the changes
print(dB_output_Pit)

{0: 0   -9.791174
Name: sig0, dtype: float64, 1: 0   -10.359401
Name: sig0, dtype: float64, 2: 0   -10.573806
Name: sig0, dtype: float64}


In [66]:
data = {'Arctic' : [dB_output_Arctic], 'Default' : [dB_output_Default]}

In [67]:
#df_pit = pd.DataFrame(dB_output_Pit)
df_Arctic = pd.DataFrame(dB_output_Arctic)
df_Default = pd.DataFrame(dB_output_Default)
df_combined = pd.concat([df_Arctic,df_Default], ignore_index = True)
df_flipped = df_combined.transpose()
df_flipped.columns = ['Arctic', 'Default']

In [68]:
print(df_flipped)

      Arctic   Default
0  -4.114165 -5.946862
1  -3.691319 -6.350994
2  -4.302779 -6.972174
3  -3.912867 -5.843008
4  -4.328355 -6.885708
5  -4.044374 -5.756196
6  -4.978822 -5.338105
7  -3.543484 -6.188963
8  -3.044157 -6.737472
9  -3.799554 -5.618316
10 -3.753096 -7.096269
11 -4.096148 -5.457205
12 -3.832138 -6.118293
13 -3.129391 -5.059742
14 -3.272850 -4.814962
15 -2.798702 -5.353105
16 -3.392685 -6.458998
17 -3.093410 -5.879387
18 -4.199101 -5.616926
19 -3.472910 -6.337872
20 -2.957259 -5.072998
21 -3.268860 -6.821397
22 -2.977067 -5.298253
23 -4.825411 -7.162023
24 -2.812081 -5.853375
25 -4.579051 -5.996093
26 -3.673281 -5.523659
27 -3.236963 -5.847367
28 -3.413143 -5.130836
29 -4.320582 -6.878418


In [69]:
df_flipped.to_csv('Bias_Correct_Backscatter_2022_2.csv', sep=';', index=False)